In [196]:
import pandas as pd
import numpy as np



In [197]:
train_data = pd.read_csv(r"C:\Users\KAVYA K\Downloads\train.csv")
test_data = pd.read_csv(r"C:\Users\KAVYA K\Downloads\test.csv", header=None)

In [198]:
train_data.head()

,Tweets,label
0,Asked #ChatGPT about what it thinks are the pr...,Positive
1,#ChatGPT tornado has already traveled around t...,Neutral
2,This is a great explanation of why #EVs are mo...,Positive
3,‘if you need to write a box-ticking social med...,Positive
4,Just saw an AI tool making my coffee for me. \...,Positive


In [199]:
train_data.shape

(770, 2)

In [200]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 770 entries, 0 to 769
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Tweets  599 non-null    object
 1   label   599 non-null    object
dtypes: object(2)
memory usage: 12.2+ KB


In [201]:
train_data.describe()

,Tweets,label
count,599,599
unique,599,3
top,Asked #ChatGPT about what it thinks are the pr...,Positive
freq,1,378


In [202]:
train_data.isnull().sum()

Tweets    171
label     171
dtype: int64

In [203]:
train_data.dropna(inplace=True)

In [204]:
data_Nan = train_data.copy(deep=True)
data_Nan[['Tweets', 'label']] = data_Nan[['Tweets', 'label']].replace(0, np.NaN)

In [205]:
train_data.duplicated().sum()

0

In [206]:
# Assuming the tweet text is in the first column
train_tweets = train_data['Tweets']
test_tweets = test_data.iloc[:, 0]

In [207]:
import re

# Data Preprocessing
def preprocess_tweet(tweet):
    # Remove special characters and links
    tweet = re.sub(r"http\S+|www\S+|https\S+", '', tweet, flags=re.MULTILINE)
    tweet = re.sub(r'\@\w+|\#', '', tweet)
    # Remove punctuation
    tweet = re.sub(r'[^\w\s]', '', tweet)
    # Convert to lowercase
    tweet = tweet.lower()
    return tweet

train_data['clean_text'] = train_tweets.apply(preprocess_tweet)
clean_test_tweets = test_tweets.apply(preprocess_tweet)

In [208]:
from keras.preprocessing.text import Tokenizer

# Tokenization
max_features = 5000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(train_data['clean_text'])
X_train_seq = tokenizer.texts_to_sequences(train_data['clean_text'])
X_test_seq = tokenizer.texts_to_sequences(clean_test_tweets)

In [209]:
from keras.preprocessing.sequence import pad_sequences

# Padding sequences
max_len = max([len(x) for x in X_train_seq + X_test_seq])
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [210]:
# Mapping sentiment to labels
#sentiment_map = {'positive': 2, 'neutral': 1, 'negative': 0}
#train_data['label'] = train_data['Tweets'].map(sentiment_map)
#test_data['label'] = test_data['Tweets'].map(sentiment_map)

In [211]:
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from keras.callbacks import EarlyStopping

#y_train = train_data['label']
#y_test = test_data['label']

# Build LSTM Model
embedding_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=X_train_pad.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [212]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Encode labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(train_data['Tweets'])

# Check the mapping of labels to integers
print(label_encoder.classes_)

# Training the model
batch_size = 32
epochs = 5
early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1)

# Assuming you have labels for your train data
y_train = train_data['Tweets'].map({'positive': 2, 'neutral': 1, 'negative': 0})


history = model.fit(X_train_pad, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_split=0.2, callbacks=[early_stop], verbose=2)

# Predict sentiment labels for test data
y_pred = model.predict_classes(X_test_pad)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)



['"#ChatGPT is not just a writing tool, it can also assist #SEO experts in keyword research, content optimization, link building, and analytics. Embrace the power of AI and take your #digitalmarketing game to the next level! #AI #SEOTools"\n\nShare\nRegenerate respo'
 '"AI has the potential to revolutionize the way crops are grown and managed" written by #ChatGPT #OpenAI https://t.co/3mdF4nLYZG'
 '"I don\'t think Web3 was that big or that metaverse stuff alone was revolutionary but AI is quite revolutionary"\n\n@BillGates is the cofounder of @Microsoft which has already invested $1 billion in #ChatGPT decrypt.co/119017/bill-ga…'
 '"I may be just a bear, but i am now part of something bigger than myself" -- what #ChatGPT thinks i\'d say if i were a bear pioneering spaceflight🧐 https://t.co/lpjkUOstpP'
 '"To err is human, to #ChatGPT is divine" - Alexander Pope (parody)'
 '"Turnitin is the go-to #software to catch students #cheating. Now it\'s focused on a potential cat-and-mouse game wi

InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\KAVYA K\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\KAVYA K\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1046, in launch_instance

  File "C:\Users\KAVYA K\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\KAVYA K\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "c:\Program Files\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  File "c:\Program Files\Python311\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "c:\Progra